# Topic Modeling the 1-Hop Monster Scrape

Build a topic model of the ~7k Wikipedia pages generated by the 1-hop spider scrape of the manually selected monster seed pages

In [1]:
spider_pages_1_dir = './spider-pages-depth-1'

To keep the token count down, we'll ignore rare words - ones that appear this many times or fewer in the corpus

In [2]:
rare_word_threshold = 50

Number of topics to model the wikipedia 1-hop data set

In [3]:
num_topics = 150

### Read the whole article text into dictionary keyed on filename

In [4]:
import os
from io import open

ext = '.txt'
file_dict = {}

txt_files = [i for i in os.listdir(spider_pages_1_dir) if os.path.splitext(i)[1] == ext]

for f in txt_files:
    with open(os.path.join(spider_pages_1_dir, f), encoding="utf-8") as file_object:
        file_dict[f] = file_object.read()
        
print ('Read ', len(file_dict), ' text files')

Read  7067  text files


In [5]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

From working with the dataset, I've learned that we should have some data-set-specific stopwords...

In [6]:
wikipedia_stop_words = {
    
    "retrieved",  # the references on each page often have a retrieval data as part of the listing
    "edit",       # the text of the label of the edit button often appears inline in the article text
    "isbn",       # appears frequenly in references
    "archived",   # the phrase 'archived from the original' appears frequently in the references
    "original",
    "url"
}

In [7]:
import re
import string

def tokenize(text):
    lda_tokens = []
    text = re.sub("http\S*", "URL", text)
    tokens = word_tokenize(text)
    for token in tokens:
        if not token.isalpha():
            continue
        #if token in string.punctuation:
        #    continue
        #if token in ["'", '"', '`', "''"]:
        #    continue
        if len(token) < 4:
            continue
        lc = token.lower()
        if lc in stop_words:
            continue
        if lc in wikipedia_stop_words:
            continue
        #if lc == 'url':
        #    continue
        lda_tokens.append(lc)
    return lda_tokens


In [8]:
# Tokenize the corpus
tokenized_docs = []
tokenized_docs_filenames = []

for f in file_dict:
    tokenized_docs_filenames.append(f)
    tokens = tokenize(file_dict[f])
    tokenized_docs.append(tokens)

In [9]:
# Peek into the tokenized data as a sanity check
print(tokenized_docs_filenames[0])
print(tokenized_docs[0])

Samuel Marinus Zwemer.txt
['american', 'missionary', 'traveler', 'scholar', 'samuel', 'marinus', 'zwemer', 'born', 'april', 'vriesland', 'michigan', 'united', 'states', 'died', 'april', 'aged', 'york', 'city', 'united', 'states', 'education', 'hope', 'college', 'brunswick', 'theological', 'seminary', 'spouse', 'elizabeth', 'wilkes', 'samuel', 'marinus', 'zwemer', 'april', 'april', 'nicknamed', 'apostle', 'islam', 'american', 'missionary', 'traveler', 'scholar', 'born', 'vriesland', 'michigan', 'received', 'hope', 'college', 'holland', 'received', 'brunswick', 'theological', 'seminary', 'brunswick', 'degrees', 'include', 'hope', 'college', 'muskingum', 'college', 'rutgers', 'college', 'ordained', 'reformed', 'church', 'ministry', 'pella', 'iowa', 'classis', 'missionary', 'busrah', 'bahrein', 'locations', 'arabia', 'member', 'arabian', 'mission', 'founder', 'american', 'mission', 'hospital', 'bahrain', 'zwemer', 'served', 'egypt', 'also', 'traveled', 'widely', 'asia', 'minor', 'elected',

In [10]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [11]:
word_freqs = dict()
for tokenized_doc in tokenized_docs:
    for word in tokenized_doc:
        if word in word_freqs:
            word_freqs[word] += 1
        else:
            word_freqs[word] = 1
            

Let's take a look at the word list prior to filtering out rare words...

In [12]:
sorted_word_freqs = sortFreqDict(word_freqs)
print('number of words: ', len(word_freqs))
sorted_word_freqs

number of words:  405820


[(71687, 'film'),
 (59745, 'also'),
 (42839, 'help'),
 (39967, 'first'),
 (37351, 'world'),
 (35949, 'target'),
 (35063, 'error'),
 (30437, 'history'),
 (29737, 'university'),
 (29457, 'time'),
 (29453, 'march'),
 (29264, 'press'),
 (29011, 'october'),
 (28610, 'series'),
 (28427, 'films'),
 (28251, 'january'),
 (27863, 'april'),
 (27665, 'july'),
 (26894, 'december'),
 (26869, 'june'),
 (26781, 'september'),
 (26325, 'august'),
 (25523, 'november'),
 (25157, 'february'),
 (24811, 'york'),
 (24415, 'united'),
 (24082, 'game'),
 (23598, 'john'),
 (23137, 'states'),
 (22515, 'used'),
 (22458, 'american'),
 (22193, 'many'),
 (21816, 'city'),
 (21509, 'would'),
 (21217, 'king'),
 (20037, 'book'),
 (19548, 'science'),
 (19362, 'godzilla'),
 (19005, 'times'),
 (18992, 'early'),
 (18866, 'years'),
 (18704, 'life'),
 (18564, 'century'),
 (18211, 'list'),
 (18122, 'article'),
 (18036, 'people'),
 (17272, 'story'),
 (17014, 'later'),
 (16792, 'news'),
 (16672, 'national'),
 (16640, 'main'),
 (16

Filter our rare words 

In [13]:
highly_filtered_tokenized_docs = []
filtered_words = set()
for tokenized_doc in tokenized_docs:
    tokens = []
    for word in tokenized_doc:
        if word_freqs[word] > rare_word_threshold:
            tokens.append(word)
            filtered_words.add(word)
    highly_filtered_tokenized_docs.append(tokens)

In [14]:
print('Vocabulary size after rare word removal:')
print(len(filtered_words))

Vocabulary size after rare word removal:
26583


In [15]:
# Peek into the tokenized data as a sanity check
print(tokenized_docs_filenames[0])
print(highly_filtered_tokenized_docs[0])

Samuel Marinus Zwemer.txt
['american', 'missionary', 'traveler', 'scholar', 'samuel', 'marinus', 'zwemer', 'born', 'april', 'michigan', 'united', 'states', 'died', 'april', 'aged', 'york', 'city', 'united', 'states', 'education', 'hope', 'college', 'brunswick', 'theological', 'seminary', 'spouse', 'elizabeth', 'wilkes', 'samuel', 'marinus', 'zwemer', 'april', 'april', 'nicknamed', 'apostle', 'islam', 'american', 'missionary', 'traveler', 'scholar', 'born', 'michigan', 'received', 'hope', 'college', 'holland', 'received', 'brunswick', 'theological', 'seminary', 'brunswick', 'degrees', 'include', 'hope', 'college', 'college', 'rutgers', 'college', 'ordained', 'reformed', 'church', 'ministry', 'pella', 'iowa', 'missionary', 'locations', 'arabia', 'member', 'arabian', 'mission', 'founder', 'american', 'mission', 'hospital', 'bahrain', 'zwemer', 'served', 'egypt', 'also', 'traveled', 'widely', 'asia', 'minor', 'elected', 'fellow', 'royal', 'geographical', 'society', 'london', 'appointed', '

In [16]:
from gensim import corpora

# Create Dictionary
id2word = corpora.Dictionary(highly_filtered_tokenized_docs)
# Create Corpus
texts = highly_filtered_tokenized_docs
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

C:\Users\peter\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 2), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 8), (24, 4), (25, 2), (26, 7), (27, 2), (28, 3), (29, 3)]


In [17]:
import gensim.models.ldamodel
from pathlib import Path
from gensim.test.utils import datapath

filename = "./lda-model"

lda_file = Path(filename)
if not lda_file.is_file():
    print("Building model")
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=1000,
                                       alpha=.1,
                                       eta=.1,
                                       update_every=0,
                                       passes=12)
    lda_model.save(filename)
else:
    print("Loading pre-built model from ", filename)
    lda_model = gensim.models.ldamodel.LdaModel.load(filename)


Loading pre-built model from  ./lda-model


In [18]:
from pprint import pprint
pprint(lda_model.print_topics(num_topics))

[(0,
  '0.012*"help" + 0.011*"target" + 0.011*"error" + 0.009*"sudan" + '
  '0.008*"aztec" + 0.007*"mexico" + 0.007*"india" + 0.006*"shinto" + '
  '0.006*"parsi" + 0.006*"religion"'),
 (1,
  '0.036*"sword" + 0.033*"bamboo" + 0.008*"used" + 0.008*"polyurethane" + '
  '0.007*"latex" + 0.007*"swords" + 0.007*"rubber" + 0.007*"foam" + '
  '0.007*"mohammad" + 0.006*"fire"'),
 (2,
  '0.053*"alien" + 0.046*"aliens" + 0.045*"predator" + 0.009*"film" + '
  '0.007*"terminator" + 0.006*"versus" + 0.006*"release" + 0.006*"marines" + '
  '0.005*"game" + 0.005*"franchise"'),
 (3,
  '0.036*"arms" + 0.019*"heraldry" + 0.009*"heraldic" + 0.008*"coat" + '
  '0.007*"also" + 0.006*"used" + 0.005*"black" + 0.005*"argent" + '
  '0.005*"gules" + 0.005*"supporter"'),
 (4,
  '0.029*"archive" + 0.027*"internet" + 0.020*"iraq" + 0.014*"library" + '
  '0.011*"steel" + 0.009*"wayback" + 0.009*"machine" + 0.006*"iraqi" + '
  '0.006*"gilgamesh" + 0.005*"march"'),
 (5,
  '0.010*"film" + 0.008*"horror" + 0.007*"januar

### Look at the top topics for each document as a sanity check

In [19]:
for filename, text in zip(tokenized_docs_filenames, highly_filtered_tokenized_docs):
    bow_doc = id2word.doc2bow(text)
    doc_to_topic = lda_model.get_document_topics(bow = bow_doc, minimum_probability = 0.05)
    print(filename)
    print(doc_to_topic)
    print()

Samuel Marinus Zwemer.txt
[(12, 0.14166567), (14, 0.113675795), (21, 0.15391138), (26, 0.12732494), (79, 0.18691991)]

P-51 Dragon Fighter.txt
[(12, 0.39546964), (79, 0.41263103)]

Wired News.txt
[(12, 0.13420483), (33, 0.05755416), (70, 0.12664588), (79, 0.1453024), (83, 0.094618276)]

Voyage to the Prehistoric Planet.txt
[(12, 0.11569323), (38, 0.2541363), (79, 0.06942161), (83, 0.18858494), (134, 0.08489533)]

Succubi in fiction.txt
[(12, 0.06910073), (26, 0.0907734), (70, 0.11562251), (90, 0.076171055), (139, 0.09645565)]

Woolworth Building.txt
[(12, 0.065255575), (46, 0.061322223), (66, 0.20318119), (70, 0.07723056), (79, 0.08911563)]

Susumu Kurobe.txt
[(66, 0.39265624), (70, 0.07566266), (79, 0.094221905), (87, 0.07230902), (102, 0.15649086)]

Motif-Index of Folk-Literature.txt
[(66, 0.55962145)]

Terminator 2-COLON- Judgment Day.txt
[(12, 0.09439039), (66, 0.2766025), (70, 0.11858048), (97, 0.086215675)]

Playmobil-COLON- The Movie.txt
[(12, 0.055011228), (62, 0.05581878), (66

The Beast (Revelation).txt
[(12, 0.071858965), (38, 0.09943942), (53, 0.07200119), (66, 0.09164945), (79, 0.09198078), (125, 0.06233886)]

Traditional animation.txt
[(66, 0.11443768), (70, 0.10459829), (86, 0.083337486), (148, 0.06650407)]

Yasuhisa Tsutsumi.txt
[(125, 0.068044476)]

Vapula.txt
[(53, 0.086682916), (66, 0.066643715), (97, 0.10141905), (125, 0.06925456)]

Sharktopus.txt
[(12, 0.07488651), (26, 0.05616114), (66, 0.117310114), (70, 0.055848297), (97, 0.10994788), (125, 0.058614686), (142, 0.08350755)]

Scarlett Johansson.txt
[(66, 0.12629005), (70, 0.0989012)]

Romance film.txt
[(66, 0.055514656), (70, 0.10437428), (79, 0.05915065), (86, 0.06480484)]

Pacific Rim (film).txt
[(12, 0.064709626), (13, 0.09142124), (26, 0.07592568), (66, 0.14307027)]

Template talk-COLON-Cloverfield.txt
[(27, 0.16028139), (86, 0.103037804), (113, 0.1991604)]

Transformers-COLON- Age of Extinction.txt
[(12, 0.089057386), (13, 0.10790821), (66, 0.08469964), (117, 0.07395495)]

Sir Walter Raleigh

Norwegian language.txt
[(12, 0.0643426), (26, 0.0523843), (53, 0.08804263), (66, 0.07458883), (77, 0.06052107), (79, 0.056342892), (86, 0.05898363)]

The Pallbearer.txt
[(12, 0.057484742), (54, 0.061016887), (66, 0.12637392), (70, 0.058316734), (79, 0.058422033)]

X2 (film).txt
[(12, 0.07357377), (66, 0.18509914), (86, 0.06500198)]

Salamander_(Dungeons_%26_Dragons).txt
[]

The H-Man.txt
[(12, 0.08463066), (26, 0.068597764), (63, 0.05134793), (66, 0.14937119), (70, 0.054328285), (86, 0.07579883)]

Shadow_(Dungeons_%26_Dragons).txt
[]

Victor Segalen.txt
[(27, 0.064065665), (70, 0.18059985), (79, 0.08352328), (131, 0.0581864)]

Philippines.txt
[(26, 0.072724625), (66, 0.09694877), (70, 0.06442024), (79, 0.06290151), (86, 0.075011164)]

Transformers (film).txt
[(12, 0.08639672), (13, 0.056447994), (26, 0.056151185), (66, 0.09976963), (97, 0.063401446), (117, 0.08383048)]

Persian Film.txt
[(63, 0.051235814), (66, 0.06388224), (70, 0.09872519), (86, 0.17126045), (97, 0.05002048), (144, 0.

Odette Yustman.txt
[(12, 0.12572758), (26, 0.15880522), (66, 0.09022392), (76, 0.09303751), (143, 0.05359837)]

Scots language.txt
[(12, 0.059552137), (64, 0.05174288), (66, 0.07825291), (76, 0.05938636), (79, 0.050081983)]

Scholia.txt
[(12, 0.057573132), (40, 0.07599245), (53, 0.0793876), (66, 0.07728211), (69, 0.07074964), (70, 0.0692345), (79, 0.061146166), (86, 0.06957612)]

Nazgûl.txt
[(12, 0.061232433), (26, 0.06708717), (53, 0.06926066), (54, 0.059571125), (66, 0.096196555), (125, 0.05702989), (139, 0.052654345)]

Template talk-COLON-Saturn Award for Best Science Fiction Film.txt
[(66, 0.05755594), (79, 0.10432698), (115, 0.08965663), (116, 0.085303545), (127, 0.07821887), (128, 0.06135557)]

Sony Electronics.txt
[(21, 0.05595067), (66, 0.10520326), (70, 0.08646979), (86, 0.05941617)]

Portal-COLON-1960s.txt
[(12, 0.057786487), (66, 0.10671467), (106, 0.07772464)]

Thai folklore.txt
[(66, 0.09127889), (74, 0.05125836), (79, 0.07791145), (97, 0.05055682)]

V-SLASH-H-SLASH-S.txt


Roc (mythology).txt
[(31, 0.07118801), (66, 0.09729669), (79, 0.06814884), (83, 0.056131016), (87, 0.07216367), (97, 0.07616068)]

Transcendence (religion).txt
[(66, 0.14857118), (79, 0.07783643)]

Saga.txt
[(26, 0.066748396), (31, 0.07305067), (66, 0.1525766), (79, 0.07206377), (86, 0.05415103)]

The Fly (2016 film).txt
[(59, 0.068633415)]

One Thousand and One Nights.txt
[(12, 0.0550266), (66, 0.14433962), (70, 0.05853709), (79, 0.08746975), (97, 0.05421953)]

New Objectivity (filmmaking).txt
[(33, 0.050174233), (63, 0.090154774), (70, 0.10612162), (86, 0.13125362), (144, 0.074212976)]

Slaughterhouse-Five (film).txt
[(12, 0.085607916), (54, 0.061069865), (63, 0.086986095), (66, 0.119796306)]

Teen film.txt
[(63, 0.059229832), (66, 0.091995046), (70, 0.09022674), (86, 0.08692546)]

Personal Computer.txt
[(26, 0.06508128), (66, 0.085285895), (70, 0.068845555), (79, 0.111952044), (86, 0.12757221)]

Sex comedy.txt
[(12, 0.06066638), (66, 0.07707809), (70, 0.07862118), (86, 0.06567641)]


...

Extinction.txt
[(66, 0.11709473), (79, 0.06289069)]

Frank Zappa.txt
[(12, 0.081599034), (66, 0.11637375), (129, 0.08555167)]

Air raid siren.txt
[(10, 0.06415259), (12, 0.07381284), (66, 0.1757779), (86, 0.09711824)]

-SLASH-Film.txt
[(12, 0.10598021), (34, 0.088040195), (51, 0.053983886), (59, 0.0747343), (66, 0.050217774), (83, 0.12779959), (86, 0.05303617), (120, 0.059979115), (139, 0.0752291)]

Ape Canyon.txt
[(64, 0.061873965), (66, 0.08787056), (79, 0.17247327), (97, 0.05162423)]

Benny Andersson.txt
[(12, 0.062089756), (21, 0.0793125), (66, 0.11288189), (79, 0.05423327), (83, 0.06938254), (93, 0.052585226)]

Alien Resurrection.txt
[(12, 0.060285315), (26, 0.07266362), (66, 0.11169891), (70, 0.085344695), (76, 0.07968766), (81, 0.050247557)]

Doug Stewart (game designer).txt
[(51, 0.059278727), (64, 0.057429086), (111, 0.06575183)]

Category-COLON-CS1 maint-COLON- unfit URL.txt
[(0, 0.05656287), (12, 0.15143214), (62, 0.06841032), (66, 0.13414824), (70, 0.13817565), (72, 0.10972

Ekecheiria.txt
[(20, 0.051873997), (34, 0.085203454), (48, 0.06350865), (86, 0.12089931), (119, 0.12902766), (127, 0.07565758)]

Brown Lady of Raynham Hall.txt
[(12, 0.100861736), (26, 0.057879433), (42, 0.060895137), (64, 0.07018693), (66, 0.13597162)]

Compact Cassette.txt
[(64, 0.08369388), (66, 0.10891168), (70, 0.07119588), (86, 0.05075414), (135, 0.08320567)]

Antihero.txt
[(12, 0.050433103), (26, 0.051372822), (66, 0.094518214), (79, 0.055878624), (97, 0.07103212)]

Coon vs. Coon & Friends.txt
[(16, 0.079196006), (66, 0.091450945), (70, 0.11388473), (79, 0.11496406), (90, 0.06547043)]

All Monsters Attack.txt
[(12, 0.05393017), (63, 0.108724914), (66, 0.14253746)]

David James (British politician).txt
[(12, 0.08163708), (66, 0.1499566), (69, 0.055834886), (73, 0.062271066), (99, 0.077965155)]

Al-Qadim.txt
[(12, 0.060688514), (66, 0.051042512), (68, 0.0974758), (70, 0.08310487), (97, 0.06755361), (125, 0.060899645), (146, 0.055917997)]

Divinity-COLON- Original Sin II.txt
[(26, 

Anatolia.txt
[(52, 0.050762273), (66, 0.07136894)]

Argonauts.txt
[(26, 0.060097147), (66, 0.06962413)]

Agdistis.txt
[(83, 0.06862759), (104, 0.06400551)]

Caanthus.txt
[(12, 0.06580325), (26, 0.112912744), (66, 0.087509885), (86, 0.10558184), (90, 0.08919056), (139, 0.100166924), (142, 0.061445143)]

Anapos.txt
[(27, 0.14091349), (31, 0.07456201), (66, 0.15751289), (70, 0.08673912), (127, 0.15029216)]

Arles.txt
[(12, 0.059615824), (66, 0.1227921), (70, 0.08520132), (97, 0.051470388)]

Bibcode (identifier).txt
[(21, 0.051940586), (53, 0.083239034), (66, 0.08046938), (79, 0.073869236), (83, 0.08017148), (93, 0.08594421), (129, 0.13517977), (134, 0.06632631)]

Arctic.txt
[(12, 0.05280687), (66, 0.09215967), (70, 0.06147269), (93, 0.11991067)]

Deep Ones.txt
[(12, 0.05779366), (13, 0.05851457), (66, 0.11248014), (70, 0.068085544)]

Aliens Online.txt
[(8, 0.10825148), (12, 0.111605704), (66, 0.1297148), (70, 0.091898575), (76, 0.08931388)]

Diphenhydramine.txt
[(12, 0.11686227), (57, 0.0

Deities & Demigods.txt
[(12, 0.054469608), (66, 0.07217625), (68, 0.06991391), (70, 0.062430106), (97, 0.067482054)]

Cretaceous–Paleogene extinction event.txt
[(66, 0.099648125), (79, 0.06946794), (97, 0.0798474)]

Escape from New York (soundtrack).txt
[(66, 0.10613326), (68, 0.057296187), (70, 0.15494862), (79, 0.08119634), (106, 0.051293425)]

Brothers Strause.txt
[(12, 0.07687489), (66, 0.1652853), (68, 0.060877547), (70, 0.057221364), (79, 0.0831877)]

Chimpanzee.txt
[(7, 0.053139254), (12, 0.06669321), (66, 0.0907382), (97, 0.066744685)]

Arcana Studio.txt
[(12, 0.05153077), (63, 0.06389355), (66, 0.0692797), (70, 0.06747558), (90, 0.12662305)]

Creation (unfinished film).txt
[(53, 0.05675046), (66, 0.09262185), (97, 0.05294938), (147, 0.1191532)]

Charles Barkley.txt
[(66, 0.07863977), (79, 0.05757043), (112, 0.11717728)]

Cigarette Burns.txt
[(26, 0.08699768), (63, 0.055345807), (66, 0.09698362), (70, 0.06916216), (117, 0.08662349)]

Agatha Christie.txt
[(60, 0.053004343), (66,

Bishop (Aliens).txt
[(8, 0.1067688), (12, 0.058646593), (26, 0.12286649), (66, 0.08320715), (76, 0.08462063)]

Charles Hill-Tout.txt
[(26, 0.15427232), (66, 0.14231881), (79, 0.05112767), (143, 0.08281838)]

Delphi.txt
[(12, 0.0551846), (26, 0.07759327), (66, 0.09382588), (70, 0.054013904), (79, 0.05157057)]

Gaasyendietha.txt
[(20, 0.05739727), (53, 0.09485372), (64, 0.08300399), (66, 0.11180662), (86, 0.11848743), (90, 0.07716718), (127, 0.0648824), (134, 0.08790271)]

Hong (rainbow-dragon).txt
[(9, 0.051459007), (26, 0.062656574), (66, 0.10579137), (97, 0.060056776)]

Fantastic art.txt
[(12, 0.116439216), (35, 0.05332979), (66, 0.10489505), (70, 0.07824264), (87, 0.06730896)]

Cantonese.txt
[(66, 0.060540475), (79, 0.050991137), (86, 0.06527956), (104, 0.08583681), (132, 0.05373973)]

Airyaman ishya.txt
[(23, 0.07154973), (66, 0.07072993), (79, 0.081448555), (122, 0.05110054), (123, 0.08003121)]

Hatching (heraldry).txt
[(12, 0.06119305), (54, 0.05987524), (66, 0.057462562), (70, 0.

### Manually chosen topics that appear to be monster-related

In [22]:
manual_monster_topics = [
2,  # alien aliens predator film terminator versus release marines game franchise
13, # greek deities children mythology zeus ancient gods also library muses
19, # loch monster ness lake water creature nessie chiang river monsters
36, # dragon dragons chinese zappa dynasty cannabis china cocaine fresno frank
41, # melville basketball silicon zombie ball whale dick herman american ahab
43, # '0.020*"whale" + 0.016*"whales" + 0.013*"press" + 0.013*"university" + '  '0.012*"hesiod" + 0.009*"west" + 0.009*"library" + 0.008*"apollodorus" + '  '0.008*"harvard" + 0.008*"titans"'),
59, # '0.039*"beowulf" + 0.012*"english" + 0.011*"norse" + 0.010*"poem" + '  '0.009*"grendel" + 0.009*"translation" + 0.007*"saga" + 0.007*"king" + '  '0.006*"verse" + 0.006*"hall"'),
60, # '0.032*"bigfoot" + 0.012*"film" + 0.012*"patterson" + 0.009*"sasquatch" + '  '0.008*"long" + 0.008*"creature" + 0.005*"krantz" + 0.005*"said" + '  '0.004*"yeti" + 0.004*"would"'),
61, # '0.013*"merlin" + 0.010*"geoffrey" + 0.010*"britain" + 0.010*"arthur" + '  '0.009*"cyclopes" + 0.008*"king" + 0.007*"haraway" + 0.006*"vortigern" + '  '0.006*"cyclops" + 0.006*"uther"'),
63, # '0.010*"dragon" + 0.009*"greek" + 0.007*"mythology" + 0.005*"also" + '  '0.005*"ancient" + 0.005*"book" + 0.004*"dragons" + 0.004*"serpent" + '  '0.003*"king" + 0.003*"name"'),
76, # '0.038*"dracula" + 0.026*"vampire" + 0.011*"sleep" + 0.010*"vampires" + '  '0.010*"frankenstein" + 0.008*"stoker" + 0.008*"rodin" + 0.007*"paralysis" + '  '0.006*"blood" + 0.006*"bram"'),
78, # '0.095*"kong" + 0.055*"king" + 0.015*"island" + 0.010*"skull" + '  '0.008*"gorilla" + 0.008*"cooper" + 0.007*"jung" + 0.007*"film" + '  '0.007*"universal" + 0.005*"jackson"'),
81, # '0.015*"heart" + 0.008*"veterinary" + 0.008*"monopoly" + 0.007*"raistlin" + '  '0.007*"swan" + 0.006*"pern" + 0.006*"mccaffrey" + 0.006*"dragon" + '  '0.005*"dragons" + 0.005*"myopia"'),
89, # '0.041*"lovecraft" + 0.017*"horror" + 0.016*"cthulhu" + 0.011*"story" + '  '0.009*"house" + 0.009*"mythos" + 0.007*"stories" + 0.007*"short" + '  '0.005*"arkham" + 0.005*"carter"'),
93, # '0.016*"cthulhu" + 0.008*"great" + 0.008*"entity" + 0.007*"mummy" + '  '0.007*"call" + 0.006*"elder" + 0.006*"year" + 0.006*"appears" + '  '0.006*"lovecraft" + 0.005*"dark"'),
101, # '0.081*"magic" + 0.026*"magical" + 0.017*"witchcraft" + 0.015*"witch" + '  '0.011*"demons" + 0.010*"spells" + 0.009*"spell" + 0.009*"witches" + '  '0.008*"divination" + 0.008*"dante"'),
104, # '0.048*"monstrous" + 0.038*"compendium" + 0.024*"appendix" + 0.022*"dragon" '  '+ 0.022*"manual" + 0.017*"monster" + 0.016*"creature" + 0.013*"volume" + '  '0.013*"giant" + 0.010*"monsters"'),
105, # '0.018*"holbein" + 0.012*"succubus" + 0.010*"nerdist" + 0.007*"portrait" + '  '0.007*"dutchman" + 0.006*"hosted" + 0.006*"flying" + 0.005*"canal" + '  '0.005*"python" + 0.005*"podcast"'),
116, # '0.024*"shark" + 0.015*"folklore" + 0.012*"fairy" + 0.012*"sharks" + '  '0.009*"water" + 0.009*"fairies" + 0.007*"also" + 0.006*"white" + '  '0.006*"irish" + 0.006*"mythology"'),
123, # '0.018*"dragons" + 0.015*"dungeons" + 0.011*"edition" + 0.008*"game" + '  '0.007*"wizards" + 0.007*"coast" + 0.007*"player" + 0.007*"dungeon" + '  '0.007*"setting" + 0.007*"campaign"'),
136, # '0.011*"scientific" + 0.011*"science" + 0.009*"paranormal" + '  '0.007*"research" + 0.005*"evidence" + 0.005*"psychic" + '  '0.005*"pseudoscience" + 0.005*"skeptical" + 0.004*"parapsychology" + '  0.004*"phenomena"'),
140, # '0.036*"ghost" + 0.025*"haunted" + 0.020*"ghosts" + 0.012*"house" + '  '0.011*"dead" + 0.007*"halloween" + 0.006*"locations" + 0.006*"paranormal" + '  '0.005*"october" + 0.005*"also"'),
143, # '0.079*"ghostbusters" + 0.010*"real" + 0.007*"aykroyd" + 0.007*"game" + '  '0.007*"slimer" + 0.007*"ghost" + 0.006*"film" + 0.006*"reitman" + '  '0.006*"video" + 0.005*"ramis"'),
145 # '0.049*"gamera" + 0.025*"list" + 0.010*"october" + 0.009*"september" + '  '0.007*"gyaos" + 0.007*"tyson" + 0.006*"flying" + 0.005*"march" + '  '0.005*"monster" + 0.004*"science"'),]
]

In [30]:
num_monster_articles = 0
for filename, text in zip(tokenized_docs_filenames, highly_filtered_tokenized_docs):
    bow_doc = id2word.doc2bow(text)
    doc_to_topic = lda_model.get_document_topics(bow = bow_doc, minimum_probability = 0.15)
    for topic in doc_to_topic:
        if topic[0] in manual_monster_topics:
            print(filename, ' - (', topic[0], ':', topic[1], ') ')
            print(lda_model.print_topic(topic[0]))
            print()
            num_monster_articles += 1

print (num_monster_articles, ' articles related to monster topics')


Wolfgang Helck.txt  - ( 123 : 0.4244498 ) 
0.018*"dragons" + 0.015*"dungeons" + 0.011*"edition" + 0.008*"game" + 0.007*"wizards" + 0.007*"coast" + 0.007*"player" + 0.007*"dungeon" + 0.007*"setting" + 0.007*"campaign"

Otia Imperialia.txt  - ( 123 : 0.21392594 ) 
0.018*"dragons" + 0.015*"dungeons" + 0.011*"edition" + 0.008*"game" + 0.007*"wizards" + 0.007*"coast" + 0.007*"player" + 0.007*"dungeon" + 0.007*"setting" + 0.007*"campaign"

Python (film).txt  - ( 123 : 0.17393818 ) 
0.018*"dragons" + 0.015*"dungeons" + 0.011*"edition" + 0.008*"game" + 0.007*"wizards" + 0.007*"coast" + 0.007*"player" + 0.007*"dungeon" + 0.007*"setting" + 0.007*"campaign"

The Dinosaur and the Missing Link-COLON- A Prehistoric Tragedy.txt  - ( 63 : 0.17320237 ) 
0.010*"dragon" + 0.009*"greek" + 0.007*"mythology" + 0.005*"also" + 0.005*"ancient" + 0.005*"book" + 0.004*"dragons" + 0.004*"serpent" + 0.003*"king" + 0.003*"name"

Ángel Sala.txt  - ( 63 : 0.20817846 ) 
0.010*"dragon" + 0.009*"greek" + 0.007*"mytholog

Template-COLON-Alien (franchise).txt  - ( 76 : 0.22145562 ) 
0.038*"dracula" + 0.026*"vampire" + 0.011*"sleep" + 0.010*"vampires" + 0.010*"frankenstein" + 0.008*"stoker" + 0.008*"rodin" + 0.007*"paralysis" + 0.006*"blood" + 0.006*"bram"

Thrillist.txt  - ( 63 : 0.21051057 ) 
0.010*"dragon" + 0.009*"greek" + 0.007*"mythology" + 0.005*"also" + 0.005*"ancient" + 0.005*"book" + 0.004*"dragons" + 0.004*"serpent" + 0.003*"king" + 0.003*"name"

Seonaidh.txt  - ( 116 : 0.24667248 ) 
0.024*"shark" + 0.015*"folklore" + 0.012*"fairy" + 0.012*"sharks" + 0.009*"water" + 0.009*"fairies" + 0.007*"also" + 0.006*"white" + 0.006*"irish" + 0.006*"mythology"

Way Out (film).txt  - ( 63 : 0.20315838 ) 
0.010*"dragon" + 0.009*"greek" + 0.007*"mythology" + 0.005*"also" + 0.005*"ancient" + 0.005*"book" + 0.004*"dragons" + 0.004*"serpent" + 0.003*"king" + 0.003*"name"

Template-COLON-The Thing (franchise).txt  - ( 81 : 0.2019524 ) 
0.015*"heart" + 0.008*"veterinary" + 0.008*"monopoly" + 0.007*"raistlin" + 0.00

Critical Role.txt  - ( 63 : 0.15795007 ) 
0.010*"dragon" + 0.009*"greek" + 0.007*"mythology" + 0.005*"also" + 0.005*"ancient" + 0.005*"book" + 0.004*"dragons" + 0.004*"serpent" + 0.003*"king" + 0.003*"name"

Fuath.txt  - ( 116 : 0.15090163 ) 
0.024*"shark" + 0.015*"folklore" + 0.012*"fairy" + 0.012*"sharks" + 0.009*"water" + 0.009*"fairies" + 0.007*"also" + 0.006*"white" + 0.006*"irish" + 0.006*"mythology"

Cebren.txt  - ( 81 : 0.15278998 ) 
0.015*"heart" + 0.008*"veterinary" + 0.008*"monopoly" + 0.007*"raistlin" + 0.007*"swan" + 0.006*"pern" + 0.006*"mccaffrey" + 0.006*"dragon" + 0.005*"dragons" + 0.005*"myopia"

Alien War.txt  - ( 76 : 0.18047182 ) 
0.038*"dracula" + 0.026*"vampire" + 0.011*"sleep" + 0.010*"vampires" + 0.010*"frankenstein" + 0.008*"stoker" + 0.008*"rodin" + 0.007*"paralysis" + 0.006*"blood" + 0.006*"bram"

Cat sìth.txt  - ( 116 : 0.17228228 ) 
0.024*"shark" + 0.015*"folklore" + 0.012*"fairy" + 0.012*"sharks" + 0.009*"water" + 0.009*"fairies" + 0.007*"also" + 0.006*"w